Whether the pressure is pinned or not, the velocity profile is identical. That's
a plus.

However, the profiles are different depending on whether the pressure is
integrated by parts or not.

So the oscillations are not caused by the convective term in the NS momentum
equation. It's definitely caused by the radial boundary condition at the axis of
symmetry.

Fantastic description of different classes of boundary conditions:
http://www.math.ttu.edu/~klong/Sundance/html/bcs.html

Well this is interesting; the _coord_type gets selected correctly when I run the
debug executable, but incorrectly when I run the opt executable.

I'm finding what Martin Leseur found: integrating p by parts creates
problems. Ok, there was a problem with the sign in the do nothing BC. When that
sign is fixed, the open flow problem is solved correctly in cartesian
coordinates whether the pressure is integrated by parts or not. The open flow
problem is solved correctly in cylindrical coordinates if the pressure is not
integrated by parts; but currently it does not converge to a solution if the
pressure is integrated by parts.

Tomorrow, record eigenvalue and flux values for different choices of the
postprocessor. Understand why the choice doesn't matter.

bxnorm = regular norm

Postprocessor Values:

| time           | bnorm          | eigenvalue     | group1max      | group1norm     | group2max      | group2norm     |
|----------------|----------------|----------------|----------------|----------------|----------------|----------------|
|   0.000000e+00 |   1.000000e+00 |   1.000000e+00 |   0.000000e+00 |   8.590069e+01 |   0.000000e+00 |   8.590069e+01 |
|   1.000000e+00 |   1.827927e+00 |   1.827927e+00 |   1.804033e-01 |   3.188402e+02 |   1.215683e-01 |   1.672426e+02 |
|   2.000000e+00 |   1.828075e+00 |   1.828075e+00 |   1.804071e-01 |   3.188541e+02 |   1.215784e-01 |   1.672560e+02 |


bxnorm = group1 norm

Postprocessor values

| time           | bnorm          | eigenvalue     | group1max      | group1norm     | group2max      | group2norm     |
|----------------|----------------|----------------|----------------|----------------|----------------|----------------|
|   0.000000e+00 |   1.164135e-02 |   1.000000e+00 |   0.000000e+00 |   1.000000e+00 |   0.000000e+00 |   1.000000e+00 |
|   1.000000e+00 |   1.049054e-02 |   1.828668e+00 |   1.034865e-03 |   1.828668e+00 |   6.967703e-04 |   9.598150e-01 |
|   2.000000e+00 |   1.048084e-02 |   1.828075e+00 |   1.034322e-03 |   1.828075e+00 |   6.970413e-04 |   9.589231e-01 |


bxnorm = group2 norm

Postprocessor Values:

| time           | bnorm          | eigenvalue     | group1max      | group1norm     | group2max      | group2norm     |
|----------------|----------------|----------------|----------------|----------------|----------------|----------------|
|   0.000000e+00 |   1.164135e-02 |   1.000000e+00 |   0.000000e+00 |   1.000000e+00 |   0.000000e+00 |   1.000000e+00 |
|   1.000000e+00 |   1.998091e-02 |   1.828113e+00 |   1.971931e-03 |   3.485134e+00 |   1.328793e-03 |   1.828113e+00 |
|   2.000000e+00 |   1.998051e-02 |   1.828075e+00 |   1.971815e-03 |   3.485013e+00 |   1.328828e-03 |   1.828075e+00 |


Notes on preconditioning:

asm-lu pre, full=false: linear its = 20
asm-lu pre, full=true: linear its = 17
default pre, full=true: linear its = 11
default pre, full=fase: linear its = 11

Ok, _grad_zero is of type std::vector<VariableGradient>. The length of the
vector is equal to the number of threads. Each element of the vector is created
like this: _grad_zero[tid].resize(getMaxQps(), RealGradient(0.));
- The type of VariableGradient is MooseArray<RealGradient>. The array has length
  equal to the number of quadrature points in the geometric element. And each
  element is of type RealGradient which is of type RealVectorValue.
- How are RealVectorValues initialized?
- RealVectorValue is of type VectorValue<Real>
- And VectorValue is a templated class

Environments for compiling on cray:

- module load cray-petsc
- module load cray-hdf5
- module switch PrgEnv-cray PrgEnv-gcc
- module load cray-mpich

So with a full preconditioner, it took 6 nonlinear iterations with PJFNK (it
does not converge with NEWTON within 50 nonlinear iterations). 7 nonlinear
iterations with just the diagonal components on the preconditioner...moreover
the number of linear iterations required was quite a bit higher. It also took 6
nonlinear iterations with line_search turned back on with PJFNK and with the
full preconditioner.

### 11/14/16

axisymm_cylinder.geo:

- H = 39.6 cm: not multiplying
- H = 80 cm; multiplying
- H = 60 cm; not multiplying
- H = 70 cm; not multiplying

There is a monotonic decrease in group 1 and group 2 fission and NSF cross sections with
increasing temperature. However, there's also a monotonic decrease in the
removal cross section as well. Diffusion constants generally trend downward with
temperature in the fuel. Same in the moderator.

Precursor1 is always the residual that causes the simulation to fail. It gets
this absolutely ridiculous depression in its concentration in the middle of the
reactor. If I look a few time steps before the simulation starts crashing, then
the precursor concentration looks like it should: monotonic. It's worthwhile to
note that precursor1 isn't the only precursor whose behavior goes from monotonic
to oscillatory; about half the precursor groups become oscillatory by the end of
failed simulations. So my job is to figure out what's going on. Why does it
crash? The problem is not:

- the RZ formulation
- the precursor decay kernels

The problem can't be fixed by:
- just reducing the advection velocity
- changing prec scaling from 1e-3 to 1e3
- increasing artificial diffusion

When the precursor simulation fails, the temperature and flux profiles look
fine.

Can definitely fix it with a sufficient source stabilization...but then the
problem becomes accuracy.

Order of tasks:

deprecated_block
finish_input_file_output
meta_action
no_action
setup_oversampling

Order of tasks that do things:
- dynamic_object_registration
    - Problem
- common_output
    - Outputs
- set_global_params
    - GlobalParams
- setup_recover_file_base
- check_copy_nodal_vars
    - temp
- setup_mesh
    - Mesh


So it makes sense that it takes so much bloody time for heat to diffuse from the
surface of the moderator into the moderator bulk. The charateristic diffusion
time from interface to wall is 42 seconds. The massive amounts of heat from
fission are being generated over the course of hundredths of a second. So yea
we're going to have to go to some wall like condition at the interface...e.g. we
cannot have Tmod = Tfuel at the interface.

Coefficient of thermal expansion, graphite: 2-6e-6 m/(m*K)

Doubling fluid flow velocity (see leastSquared directory), had absolutely no
impact on the maximum temperature in the reactor which seems a little bit odd to
me.

Putting those dumb boundary conditions in place for the temperature in the
moderator produces a hideous spectrum. I think how Cammi got around these things
is because he never used local temperatures to determine his properties. He used
block averaged values. So he could integrate over half the moderator domain to
determine the average temperature and then use that for the neutron group
constants in the other half.

Working on monotone cubic interpolation:

The second order accurate finite difference methods for derivative computation
will compute derivatives exactly for a second order polynomial. They WILL NOT
correctly compute derivatives for a third order polynomial because they are only
second order accurate. The FC algorithm initializes derivatives using this
second order differencing scheme. However, the FB algorithm roughly averages
neighboring secant lines...this is not second order accurate (or is it?).

Let's say we have some function. We want to evaluate/estimate it's derivative at
a point. How can we do that? We can perform a finite difference to do the
estimation:

f'(x) ~ (f(x+h) - f(x-h)) / h

We can write the error of this approximation as:

E(h) = C * h**n

where n is the order of accuracy. We can imagine a case where a method is nth
order accurate and the error is zero; this corresponds to C = 0.

From wikipedia: "The Newton series consists of the terms of the Newton forward
difference equation, named after Isaac Newton; in essence, it is the Newton
interpolation formula, namely the discrete analog of the continuum Taylor
expansion, which holds for any polynomial function f and for most (but not all)
analytic functions.

You can define your numerical derivative however you want! But then the key is
you need to be able to set-up an expression:

y' + Error(stuff, h) = numerical\_derivative

And then you can inspect the parts that make up the error in order to determine
it's functional dependence on h and perhaps whether the error is zero because of
other components that make up the error function. Cool!

yt is not correctly rendering any of my new neutronic simulation data sets! Need
to figure out why.

Ok first test was running a single region test with Quad4 elements. That test
went sucessfully which makes sense because Andrew has had a Q2 sampler
implemented forever.

It should be noted that data values in yt aren't actually accessed until
performing data access, something like `ad['diffused']`. Even this operation:
`ad = ds.all_data()` won't actually read the variable values.

Prism volumes:
second order volumes: 1.17718
first order volumes: 1.06066

For just one element:
first order volume: 1.5 (as it should be)
second order volume: 2.32843; very close to 2.356 which is equal to:
pi * r^2 * h / 4

The rule for computing the PRISM18 volume can be found in the libmesh source.

The natural boundary condition appears to have an effect on the flow at the
boundary for shizzle.

I've been able to get the flow simulation to work perfectly with the correct
geometry and the correct boundary conditions. Nice!

Ok, so when linking a library with -l, the linker searches in order from first
to last of directories provided with -L. -rpath matters for run-time; if there's
a mistake in supplying rpath during the linking stage, this error will show up
at run-time, NOT during program linking.

ldd uses LD_LIBRARY_PATH. You can kind of think of ldd as a command that
executes the shared object in order to discover its dependencies. Things that
get executed care about their environment; e.g. in this case ldd certainly cares
about LD_LIBRARY_PATH.

Changing LD_LIBRARY_PATH will change the memory location/library version that a
shared object points to. E.g. if I alter LD_LIBRARY_PATH, then I can change the
location of the library that libgfortran.so.3 points to.

When resolving shared object dependencies of an executable or other shared
object, the looking order is roughly the following:

1. Directories specified in the DT_RPATH dynamic section attribute of the
   executable/shared object in question (this is set at compile/linking time for
   example with -W,-rpath)
2. From the environment variable LD_LIBRARY_PATH
3. From the library directories listed in /etc/ld.so.conf
4. Finally in /lib and then /usr/lib

Nice!

The problematic file appears to be...kind of the core shared
object...libmesh_dbg.so

Successfull compilation command for the previously failing unit_tests-dbg:

ccache clang++ -std=gnu++11 -O0 -felide-constructors -g -pedantic -W -Wall
-Wextra -Wno-long-long -Wunused -Wpointer-arith -Wformat -Wparentheses
-Qunused-arguments -Woverloaded-virtual -fopenmp -std=gnu++11 -o
.libs/unit_tests-dbg hello.cpp -Wl,-rpath -Wl,/opt/moose/tbb/lib -Wl,-rpath
-Wl,/opt/moose/petsc/mpich_petsc-3.7.4/clang-opt-superlu/lib -Wl,-rpath
-Wl,/opt/moose/gcc-6.2.0/lib/gcc/x86_64-pc-linux-gnu/6.2.0 -Wl,-rpath
-Wl,/opt/moose/gcc-6.2.0/lib64 -Wl,-rpath -Wl,/lib/x86_64-linux-gnu -Wl,-rpath
-Wl,/opt/moose/gcc-6.2.0/lib -Wl,-rpath
-Wl,/opt/moose/mpich/mpich-3.2/clang-opt/lib -Wl,-rpath
-Wl,/opt/moose/llvm-3.9.0/lib -Wl,-rpath -Wl,/usr/lib/x86_64-linux-gnu
-L/opt/moose/petsc/mpich_petsc-3.7.4/clang-opt-superlu/lib
-L/opt/moose/mpich/mpich-3.2/clang-opt/lib -L/opt/moose/gcc-6.2.0/lib64
-L/opt/moose/gcc-6.2.0/lib
/home/lindsayad/test_multiple_petsc/libmesh/build/contrib/netcdf/v4/liblib/.libs/libnetcdf.so
/usr/lib/x86_64-linux-gnu/libcurl-gnutls.so -lz -L/opt/moose/tbb/lib -ltbb
-ltbbmalloc -lpetsc -lsuperlu_dist -lcmumps -ldmumps -lsmumps -lzmumps
-lmumps_common -lpord -lparmetis -lmetis -lHYPRE
-L/opt/moose/gcc-6.2.0/lib/gcc/x86_64-pc-linux-gnu/6.2.0 -L/lib/x86_64-linux-gnu
-L/opt/moose/llvm-3.9.0/lib -lscalapack -lflapack -lfblas -lX11 -lhwloc
/opt/moose/mpich/mpich-3.2/clang-opt/lib/libmpifort.so
/opt/moose/gcc-6.2.0/lib/../lib64/libgfortran.so
/opt/moose/gcc-6.2.0/lib/../lib64/libgomp.so
/opt/moose/gcc-6.2.0/lib/../lib64/libquadmath.so
/opt/moose/mpich/mpich-3.2/clang-opt/lib/libmpicxx.so
/opt/moose/gcc-6.2.0/lib/../lib64/libstdc++.so -lm
/opt/moose/mpich/mpich-3.2/clang-opt/lib/libmpi.so -lrt -lomp -lgcc_s -lpthread
-L/usr/lib/x86_64-linux-gnu -L/opt/moose/cppunit-1.12.1/clang-opt/lib -lcppunit
-ldl -pthread -fopenmp -Wl,-rpath
-Wl,/home/lindsayad/test_multiple_petsc/scripts/../libmesh/installed/lib
-Wl,-rpath -Wl,/opt/moose/gcc-6.2.0/lib/../lib64 -Wl,-rpath
-Wl,/opt/moose/mpich/mpich-3.2/clang-opt/lib

Should it be enough...?

I learned exactly why the libmesh build was failing. The reason is that when
compiling and linking the libmesh_dbg library, the /usr/lib/x86_64-linux-gnu
directory was passed to the linker before the opt petsc directory, meaning that
-lpetsc links in the system petsc as opposed to the MOOSE environment petsc. And
then when compiling and linking the unit_tests-dbg shared object, the opt petsc directory
is passed first, so there's a mismatch there.

To summarize: system petsc can be used to build libmesh/moose as long as the
moose environment is not loaded. However, it's pretty much impossible to unload
the system packages, so the moose environment petsc cannot be used to build
libmesh/moose as long as the system petsc is installed.

Knowing why this failed is enough for me. There's really no practical use to
having two petsc installations loaded at any time.

Cannot source .bashrc when a conda virtualenv is active.

In typedef:

typedef <ITEM1> <ITEM2>

The second item is being defined to equal the first item, e.g., you can think of
it as:

ITEM2 = ITEM1

e.g. ITEM1 is the thing that already exists.

Looking at dof_map_constraints.C in libMesh:

During interpolation of the nodes, these dofs get fixed:

first element (0xa74eb0):
0
1
3
4
7
This must be a corner element of the global mesh

second element (0xa75020):
0
1
4
This must be a side element of the global mesh

So question is: how is it determined whether a degree of freedom is nodal or edge? A reasonable guess would be that a "nodal" degree of freedom corresponds to an element vertex.

Another question: why is it that on edge dofs the solution values aren't simply assigned to the evaluation of the dirichlet function?

On a quadrilateral side (with second order shape functions), there are three quadrature points. I'm curious where the quadrature points are located.

Ok, stripping the problem down to one element. Then the quadrature points of one example side are:

(-.77, -1, 0)
(0, -1, 0)
(.77, -1, 0)

Which of these also corresponds to a degree of freedom?? Why the point (0, -1, 0) does of course.

Solution value at (-.77, -1, 0): -.3467

Ok the solution values at the quadrature points calculated in libmesh match the dirichlet function being projected onto them. That's good!

I need to understand what an infinite homogeneous medium really means in the context of Scale.

Files that cause compilation errors during Scale build:

packages/Tsurfer/read_npt.f90:1552
packages/Senlib/read_response_M.f90:277
packages/Senlib/read_response_M.f90:394

Trying to get familiar with Scale execution. Currently performing an infinite lattice calculation with NEWT (sample newt1.inp).

Call sequence so far:

main (ScalePrograms/scale.cpp)
Scale::ScaleDriver::run (ScaleDriver/ScaleDriver.cpp)
Scale::ScaleDriver::runSequence (ScaleDriver/ScaleDriver.cpp)
Scale::Sequence::TNewt::execute (Sequence/TNewt/TNewt.cpp)
Scale::Module::BaseTMGTransport::execute (Module/Xsdrn/BaseTMGTransport.cpp)
Scale::Module::XSProcModule::execute (Module/XSProc/XSProcModule.cpp)

From whithin XSProceModule::execute we call:

Crawdad::execute (Crawdad/Crawdad.cpp)

amd then:

XSProc::execute (XSProc/XSProc/XSProc.cpp)

From within XSProc::excute we call:

Dancoff::execute (XSProc/dancoff/Dancoff.cpp)
execute_dancoff (XSProc/dancoff/DancoffInterface.cpp)
F_executeDancoff (XSProc/dancoff/Dancoff_I.f90)
execute_dancoff (XSProc/dancoff/Dancoff_M.f90)

and then:

bonamiM::execute (XSProc/bonamiM/nonamiM.cpp)

and then:

MixMacros::execute (XSProc/MixMacros/MixMacros.cpp)

and then:

CentrmPmc::execute (PMC/CentrmPmc.cpp)

and then:

MixMacros::execute

Once all that is finished we return to Scale::Module::BaseTMGTransport::execute (BaseTMGTransport.cpp) which then calls:

Scale::Module::NewtModule::execute (Module/Newt/NewtModule.cpp)

BaseTMGTransport::execute looks like an important function. It is the function from which both the execution routines for generating the multigroup cross sections and the actual transport code are called.

Here's Newt backtrace:

#0  newtapifortran_m::executenewt (inputsaver=..., lib=...)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Newt/NewtApiFortran.f90:184
#1  0x00007fffe8673d47 in newtapibinding_m::f_executenewt (
    inputsaver=<error reading variable: Attempt to dereference a generic pointer.>,
    libs=<error reading variable: Attempt to dereference a generic pointer.>)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Newt/NewtApiBinding.f90:38
#2  0x00007ffff2b88d0b in NEWT::NewtApi::executeNewt (this=0x7fffffffc130, inputSaver=0x5db0718, lib=0x238af770)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Newt/NewtApi.h:41
#3  0x00007ffff2b85689 in Scale::Module::NewtModule::execute (this=0x5db0660)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Module/Newt/NewtModule.cpp:212
#4  0x00007ffff293932b in Scale::Module::BaseTMGTransport::execute (this=0x68f710)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Module/Xsdrn/BaseTMGTransport.cpp:174
#5  0x00007ffff7206c94 in Scale::Sequence::TNewt::execute (this=0x68f470)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/Sequence/TNewt/TNewt.cpp:200
#6  0x00007ffff7bc16a3 in Scale::ScaleDriver::runSequence (this=0x7fffffffd620, name=..., sequence=..., current_sequence_stream=...)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/ScaleDriver/ScaleDriver.cpp:591
#7  0x00007ffff7bbfe6c in Scale::ScaleDriver::run(Standard::Factory<Standard::AbstractSequence*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::function<Standard::AbstractSequence* ()> >&) (this=0x7fffffffd620, sequenceFactory=...)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/ScaleDriver/ScaleDriver.cpp:389
#8  0x000000000041919f in main (argc=2, argv=0x7fffffffd908)
    at /home/lindsayad/scale/SCALE-6.2-serial-6.2.1-Source/packages/ScalePrograms/scale.cpp:173

Now looking at executing my silly little input file.

Still call Crawdad and then again:

Dancoff
Bonami
MixMacros
CentrmPmc
MixMacros

Drilling down a little into centrmpmc.

Crawdad gets called according to the documentation because CentrmPmc requires it (or something like that).

Ok, using an infinite lattice as opposed to inifinite homogeneous medium, I still get high k values:

k-eff = 1.41823805
k-infinity from four factor formula = 1.419670

After "Critical Spectra Calculated with the B1 Method":

Critical Buckling = 1.05846e-3
k-infinity of critical system = 1.38594

Changing the grid size from 5x5 to 10x10 (at least I believe that's what I was doing) did not appreciably change the results.


## 1/11/17

Trying to understand what nodal methods means. Here are some excerpts from <i>Progress in Nodal Methods for the Solution of the Neutron Diffusion and Transport Equations</i> written by Lawrence, published in 1985:

> "Practical limiations on computer storage and execution time generally prohibit the explicit modeling of each fuel pin in a LWR. Instead 'equivalent' few-group diffusion-theory parameters are determined for relatively large homogeneous regions often consisting of entire fuel assemblies in the radial plane. With these parameters in hand, global solutions are computed for this homogenized-assembly representation of the reactor. Solution of this problem using traditional finite-difference techniques requires a large number of mesh points in orrder to represent accurately the spatial variation of the neutron flux. The computational expense associated with these calculations motivated the early development of less rigorous, yet more computationally efficient techniques oriented towards the determination of the flux averaged over each homogeneous region or 'node'. This class of methods thus became known as nodal methods, and the FLARE model developed in 1964 is representative of the first generation of these schemes."


In [2]:
sigma_t_1 = 3.05946e-1
sigmas_1to1 = 2.94802e-1
sigmas_1to2 = 1.94277e-3
sigmas_1 = sigmas_1to1 + sigmas_1to2
sigma_a_1 = sigma_t_1 - sigmas_1
print(sigmas_1)
print(sigma_a_1)

0.29674477
0.009201230000000005


In [4]:
sigma_t_2 = 3.15036e-1
sigma_s_2to1 = 1.58033e-3
sigma_s_2to2 = 2.76692e-1
sigma_s_2 = sigma_s_2to1 + sigma_s_2to2
sigma_a_2 = sigma_t_2 - sigma_s_2
print(sigma_s_2)
print(sigma_a_2)


0.27827233
0.03676366999999997


Alright, confirmed that we can deduce the `Total- Scatter` cross section (equivalent to what I am used to thinking of as the absorption cross section) from summing the contributions of the P0 scattering matrix. Good!

## 1/12/17

Trying to figure out how to pass default coupling values through to actions:

```C++
void
InputParameters::addCoupledVar(const std::string & name, const std::string & doc_string)
{
  addParam<std::vector<VariableName> >(name, doc_string);
  _coupled_vars.insert(name);
}

Ok, we're passing T = std::vector<VariableName> >

template <typename T>
void
InputParameters::addParam(const std::string &name, const std::string &doc_string)
{
  checkConsistentType<T>(name);

  InputParameters::insert<T>(name);
  _doc_string[name] = doc_string;
}

Recall again, T = std::vector<VariableName> >

template <typename T>
inline
void Parameters::insert (const std::string & name)
{
  if (!this->have_parameter<T>(name))
    _values[name] = new Parameter<T>;

  set_attributes(name, true);
}

Note that the if block in the above function does get entered as one might hope.

So in compilation, what does that code pattern look like?

  checkConsistentType<T>(name);
  
  if (!this->have_parameter<T>(name))
    _values[name] = new Parameter<T>;

  set_attributes(name, true);
  _doc_string[name] = doc_string;
  
  _coupled_vars.insert(name);  
  
Hmm, that's interesting, here's what InputParameters::set looks like:

  checkConsistentType<T>(name);

  if (!this->have_parameter<T>(name))
    _values[name] = new Parameter<T>;

  set_attributes(name, false);

  if (quiet_mode)
    _set_by_add_param.insert(name);

  return cast_ptr<Parameter<T>*>(_values[name])->set();
  
```

It's important to note that we are setting to a `getParam`

```
getVecMooseTypes searches itself for a parameter with name "name". If it finds that parameter, then it tries to get the vector of variable names accessible by name. 

Parameters::_values is a map. It's like a python dictionary. It's keys are the parameter names. And then each key has an associated value which can be of a variety of types. Apparently the key "temperature" is associated with a vector of VariableNames. And in the normal Kernel case, when acessing the value corresponding to the key name of "v", the value returned is essentially NULL. How come that doesn't happen for me??? My question is: when does the value part of the key-value pair get added to the map??

In Coupleable, we loop from _c_parameters.coupledVarsBegin() to _c_parameters.coupledVarsEnd() which just inspects _coupled_vars. _coupled_vars itself is set of strings, e.g. it's a set of the coupled variable names.

So as already discussed, one of my big issues is that I have no idea at what point the value part of the key-value pair for "temperature" gets created.
```

```
Apparently cast_ptr is a type definied by libmesh. Knew it!
```

```
Here's default temperature of 800:

+----------------+----------------+----------------+----------------+
| time           | group1_current | group1_old     | multiplication |
+----------------+----------------+----------------+----------------+
|   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |
|   1.000000e-04 |   3.539391e+03 |   1.328831e+03 |   2.663538e+00 |
|   2.200000e-04 |   3.186612e+03 |   3.539391e+03 |   9.003279e-01 |
|   3.640000e-04 |   2.307495e+03 |   3.186612e+03 |   7.241216e-01 |
|   5.368000e-04 |   1.524371e+03 |   2.307495e+03 |   6.606172e-01 |
|   7.441600e-04 |   9.246172e+02 |   1.524371e+03 |   6.065567e-01 |
|   9.929920e-04 |   5.076227e+02 |   9.246172e+02 |   5.490085e-01 |
|   1.291590e-03 |   2.472787e+02 |   5.076227e+02 |   4.871309e-01 |
+----------------+----------------+----------------+----------------+

Here's default temperature of 1050:

Postprocessor Values:
+----------------+----------------+----------------+----------------+
| time           | group1_current | group1_old     | multiplication |
+----------------+----------------+----------------+----------------+
|   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |
|   1.000000e-04 |   3.539391e+03 |   1.328831e+03 |   2.663538e+00 |
|   2.200000e-04 |   3.186612e+03 |   3.539391e+03 |   9.003279e-01 |
|   3.640000e-04 |   2.307495e+03 |   3.186612e+03 |   7.241216e-01 |
|   5.368000e-04 |   1.524371e+03 |   2.307495e+03 |   6.606172e-01 |
|   7.441600e-04 |   9.246172e+02 |   1.524371e+03 |   6.065567e-01 |
|   9.929920e-04 |   5.076227e+02 |   9.246172e+02 |   5.490085e-01 |
|   1.291590e-03 |   2.472787e+02 |   5.076227e+02 |   4.871309e-01 |
+----------------+----------------+----------------+----------------+

The exact frieking same.

With the default temperatures changed in the kernel source files:

Postprocessor Values:
+----------------+----------------+----------------+----------------+
| time           | group1_current | group1_old     | multiplication |
+----------------+----------------+----------------+----------------+
|   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |   0.000000e+00 |
|   1.000000e-04 |   3.539391e+03 |   1.328831e+03 |   2.663538e+00 |
|   2.200000e-04 |   3.186612e+03 |   3.539391e+03 |   9.003279e-01 |
|   3.640000e-04 |   2.307495e+03 |   3.186612e+03 |   7.241216e-01 |
|   5.368000e-04 |   1.524371e+03 |   2.307495e+03 |   6.606172e-01 |
|   7.441600e-04 |   9.246172e+02 |   1.524371e+03 |   6.065567e-01 |
|   9.929920e-04 |   5.076227e+02 |   9.246172e+02 |   5.490085e-01 |
|   1.291590e-03 |   2.472787e+02 |   5.076227e+02 |   4.871309e-01 |
+----------------+----------------+----------------+----------------+

The exact frieking same. There's some weird stuff going on right now, and I'm too tired to debug it.

Well I figured out why this wasn't changing...the temperature is not used in any of the kernel calculations. It's only used in the material calculation and that has a default setting itself which was totally unaffected by any temperature changes I was trying to affect in the action.
```

I'm a glutton for punishment, what can I say.

```
params.addCoupledVar gets called before InputParameters::defaultCoupledValue which actually sets the default coupled value.

Here's the latter's backtrace:

#0  InputParameters::defaultCoupledValue (this=0xa2db98, coupling_name=..., value=3)
    at /home/lindsayad/projects/moose/framework/src/utils/InputParameters.C:426
#1  0x00007ffff66d9c57 in Parser::setVectorParameter<VariableName> (this=0x9db9e0, full_name=..., short_name=..., param=0xa30070)
    at /home/lindsayad/projects/moose/framework/src/parser/Parser.C:1291
#2  0x00007ffff66d622a in Parser::extractParams (this=0x9db9e0, prefix=..., p=...)
    at /home/lindsayad/projects/moose/framework/src/parser/Parser.C:832
#3  0x00007ffff66ce6cf in Parser::parse (this=0x9db9e0, input_filename=...)
    at /home/lindsayad/projects/moose/framework/src/parser/Parser.C:246
#4  0x00007ffff5d314e9 in MooseApp::setupOptions (this=0x9dab60) at /home/lindsayad/projects/moose/framework/src/base/MooseApp.C:327
#5  0x00007ffff5d3382d in MooseApp::run (this=0x9dab60) at /home/lindsayad/projects/moose/framework/src/base/MooseApp.C:575
#6  0x0000000000424449 in main (argc=3, argv=0x7fffffffd8a8) at /home/lindsayad/projects/moose/test/src/main.C:35
```


## 1/13/17

Looking at some integer arithmetic

In [1]:
bits = [i for i in range(32)]
sum = 0
for bit in bits:
    sum += 2**bit
print(sum)

4294967295


Above is maximum representable integer value with 32 bits. Cool!

I want to show a sequence of backward stepping by gdb.

Set breakpoint at Parser.C:704, running `gdb --args ../../../moose-test_dbg -i optionally_coupled.i` in `moose/test/tests/variables/optionally_coupled/`:

```
bool found = false; 
bool in_global = false;                                                                                                 │
std::string orig_name = prefix + "/" + it.first;                                                                        │
std::string full_name = orig_name;
    if (_getpot_file.have_variable(full_name.c_str()) || (_app.commandLine() && _app.commandLine()->haveVariable(full_name.c_str())))
        else if (global_params_block != NULL)
            if (!found)
```

Currently, I'm getting a critical reactor with serpent input for height=396 and nt_scale=1e0. Getting a sub-critical reactor with newt input for height=396 and nt_scale=1e16. Going to try the latter with nt_scale=1e0 when I get back from the run. Also with nt_scale=1e0 no criticality

- The group 2 NSF value (fissxs * numbar) is a little lower in the fuel for newt than it is for serpent.
- The diffusivity of group 2 in the fuel is about twice as high with serpent as it is with NEWT
- Scattering from group 2 to 1 in the fuel quite a bit higher in newt than in serpent
- Remxs from group 1 in moderator quite a bit higher in newt than serpent
- Ok, here are the first order of magnitude differences I've seen: at 800 K in moderator:
NEWT: g1 D = .991; g2 D = .776
SERPENT: g1 D = .35; g2 D = .02
- The group 2 diffusivity in newt is like 40 times higher!!!

Ok even swapping the diffusivity coefficients in the moderator from newt's to serpent's did not make the reactor critical. If I inspect the newt_moltres output file after 18 time steps, the shape of the neutron distributions are the same as all the neutron distributions I've seen while using Serpent input data.

So question: why does newt say a critical buckling occurs at 115 cm when I can't use it's collapsed group xsecs to make a critical reactor at 396 cm!?!?!?

With serpent inputs, geometry is sub-critical at 115 cm but super-critical at 200 cm. Subcritical at 150. Supercritical at 175. So we know critical buckling occurs with serpent inputs for 150 < H < 175 (cm).

With newt inputs, gemoetry still sub-critical at 500 cm. So different from the actual newt calculation!!!

### So awesomely, the order of scattering is reversed from Serpent to Newt.

- Serpent does P(1 -> 1), P(2 -> 1), ...
- Newt does (what I think is more sensical) P(1 -> 1), P(1 -> 2), ...

Hey, what do you know? Transposing the scattering matrix made the newt reactor super critical at 396 cm!!! That's a start! And at 200! Sub-critical at 100. Super Critical at 150! And at 125. Very slightly super critical at 115! Sub at 110. Sub at 112. Sub at 114.

# (perfectly_bracketed_critical_buckling_height_from_newt): So this is pretty freaking awesome

## This moltres problem directory just considers neutronics, e.g. no temperature coupling

For 908 K in newt with MSBR concentrations, the predicted critical buckling factor was 7.00660e-4 corresonding to a reactor height of 114.76 cm. We then ask newt to output collapsed two group constants. We feed those group constants into Moltres (at admittedly 900 K). Results:

## At 114 cm, the reactor is sub-critical

## At 115 cm, the reactor is super-critical

How about that for computational science?! Maybe I can still run simulations after all!

For the record [114, 115] is still the bracket for 908 K in Moltres :-) And increasing the Moltres temperature to 950 K leads to a sub-critical reactor at 115 cm.

# (add_temperature_coupling): Now coupling back in temperature

So now continuing to use newt material values and a reactor height of 115 cm, I run a simulation coupling back in temperature with an inlet value of 824 K. We produce a nice critical reactor with outlet temperature of 992 K and tradtional neutronic distribution centered in the axial center of the reactor, and thermal group concentrated in the moderatore region and fast group concentrated in the fuel region.

### Important note

In order to get this nice criticial reactor, had to cut time step growth factor from 1.2 to 1.05. Otherwise I observe the same phenomenon I've seen in the past: at some arbitrary time step the neutron population will suddenly decrease by many orders of magnitude and the reactor will go the a uniform temperature of 824 K with no neutron population.

## Note that above two cases do not include precursors --> That will be next on the docket